In [2]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
df = pd.read_csv("../INPUT/stock_data.csv") 

In [7]:
# Checking to see if the dataset needs any cleaning, but it doesn't:
print(df.isnull().sum())
print(df.duplicated().sum())


Text         0
Sentiment    0
dtype: int64
0


In [8]:
# Web scraping to get the symbols for all the stocks on the S&P500
res = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies").text
soup = BeautifulSoup(res)
sp500 = []
for n in range(2, 506):
    sp_ticker = soup.select(f"#constituents > tbody > tr:nth-child({n}) > td:nth-child(1) > a")[0].text
    sp500.append(sp_ticker)

KeyboardInterrupt: 

In [ ]:
print(sp500)

In [ ]:
"""
Now, I want to add another column to the data set, with the ticker or name of the security:
- Look for #
- Look for capitalized letters -- Compare to another sp500 (which is the list I got from web scraping the wikipedia page) 
"""

df["Ticker"] = 0  #Intializing column

for e in range(df["Text"].count()):
    final_list = []
    list_tickers = (re.findall(r'\b([A-Z]+[A-Z])\b', df["Text"][e]))

    for ticker in list_tickers:
        if ticker in sp500:
            final_list.append(ticker)
            df["Ticker"][e] = final_list[0]
    
    if len(final_list) == 0:
        df["Ticker"][e] = None
        
    


df.head(50)
df.isnull().sum()
df_clean = df.dropna()

df_clean


#def extractSecurity (text):


In [ ]:
df_clean['Ticker']=df_clean['Ticker'].replace("AAP","AAPL")
df_clean.loc[df_clean['Ticker']=='AAPL']


In [10]:
df_clean['Positive Sentiment']=df_clean['Sentiment'].replace(-1,0)
df_clean['Negative']=df_clean['Sentiment'].replace(1,0)
df_clean['Negative Sentiment']=df_clean['Negative'].replace(-1,1)


#Dropping of the temporary column 'Negative'
df_clean=df_clean.drop('Negative', axis=1)
#Dropping of the Sentiment column
df_clean=df_clean.drop('Sentiment', axis=1)
df_clean

            

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,Text,Ticker,Positive Sentiment,Negative Sentiment
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,AAPL,1,0
2,user I'd be afraid to short AMZN - they are lo...,AMZN,1,0
6,AAP - user if so then the current downtrend wi...,AAPL,0,1
7,Monday's relative weakness. NYX WIN TIE TAP IC...,TAP,0,1
8,GOOG - ower trend line channel test & volume s...,GOOG,1,0
...,...,...,...,...
5649,"Sensex Ends 693 Points Higher, Nifty Reclaims ...",IT,1,0
5650,"Sensex closes 693 points higher, Nifty at 7,80...",IT,1,0
5684,"Sensex jumps over 750 points to cross 29,000 m...",IT,1,0
5772,"Sensex Opens 166 Points Lower, Nifty Starts Da...",IT,0,1


In [11]:
df_grouped = df_clean.groupby("Ticker").agg(
    positive_sentiment = ("Positive Sentiment", sum),
    negative_sentiment = ("Negative Sentiment", sum))

df_grouped.reset_index(level=0, inplace=True)
type(df_grouped)
df_grouped.count()

Ticker                183
positive_sentiment    183
negative_sentiment    183
dtype: int64

In [9]:
date = "2020-06-19"
key = os.getenv("ALPHAVANTAGE_APIKEY")






In [13]:
ts = TimeSeries(key)

df_grouped["low_price"] = 0
df_grouped["high_price"] = 0

for e in range(df_grouped["Ticker"].count()):
    low, meta = ts.get_weekly(symbol= df_grouped["Ticker"][e])
    df_grouped["low_price"][e] = low[date]["3. low"]
    high, meta = ts.get_weekly(symbol= df_grouped["Ticker"][e])
    df_grouped["high_price"][e] = high[date]["2. high"]
  

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,Ticker,positive_sentiment,negative_sentiment,low_price,high_price
0,AAPL,461,373,332.5800,356.5600
1,ABBV,1,0,90.7800,98.2000
2,ABC,1,0,92.9000,103.3700
3,ABMD,1,0,236.3500,260.4500
4,ABT,4,0,87.0500,92.4000
...,...,...,...,...,...
178,WMB,2,0,17.4800,19.9300
179,WMT,9,11,117.0100,120.3000
180,WYNN,5,4,86.0400,98.1500
181,XOM,1,1,44.7500,49.7500


In [15]:
df_grouped.to_csv("df_analysis.csv", index = False)

NameError: name 'df_grouped' is not defined